In [3]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
from os.path import dirname, join

In [4]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

def show_image(img, name: str = "Image"):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def write_image(img: np.array, path: str):
    cv2.imwrite(path, img)
    
def get_chessboard_points(chessboard_shape, dx, dy):
    # Get number of corners in each dimension
    rows, cols = chessboard_shape
    
    # Create a list to store 3D coordinates
    object_points = []
    
    # Generate 3D coordinates for each corner
    for i in range(cols):
        for j in range(rows):
            # X corresponds to column (j), Y corresponds to row (i), Z is always 0
            object_points.append([j * dx, i * dy, 0.0])
    
    # Convert to numpy array with float32 dtype
    return np.array(object_points, dtype=np.float32)

In [ ]:
def main(video_source, width=1280, height=720):
    # Inicializar la captura con el backend FFMPEG (recomendado para IP cams)
    cap = cv2.VideoCapture(video_source, cv2.CAP_FFMPEG)
    
    # Reducir el buffer al mínimo para evitar retrasos en la transmisión en vivo
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    if not cap.isOpened():
        print(f"No se pudo conectar a la fuente: {video_source}")
        return

    window_name = "Cámara IP"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Conexión perdida o frame no recibido.")
                break

            # Redimensionar la imagen al tamaño deseado
            frame = cv2.resize(frame, (width, height))

            cv2.imshow(window_name, frame)

            # Salir con la tecla 'q' o ESC
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q') or key == 27:
                break
                
    except KeyboardInterrupt:
        pass
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Asegúrate de que tu IP y puerto sean correctos
    url_camara = "http://192.168.1.44:8080/video"
    
    main(video_source=url_camara)

In [3]:
path = join(os.getcwd(), "data", "calibration", "chessboards")

imgs_path = [join(path, f"{img_path}") for img_path in os.listdir(path)]
imgs = load_images(imgs_path)

C:\Users\juan\AppData\Local\Temp\ipykernel_24500\2034268774.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [4]:
corners = [cv2.findChessboardCorners(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), (8,6)) for img in imgs]

corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]
# [show_image(img) for img in imgs_gray]
corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

output_folder = "data/calibration/outputs/"

imgs_copy = copy.deepcopy(imgs)
imgs_with_points = [cv2.drawChessboardCorners(img, (8,6), corner, True) for img, corner in zip(imgs_copy, corners_refined)]
# Check the folder exists
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

# Show and save
for i, img in enumerate(imgs_with_points):
    show_image(img)
    output_img_path = join(output_folder, f"detection_{i}.png")
    write_image(img, output_img_path)

In [ ]:
chessboard_points = np.array([get_chessboard_points((8, 6), 29, 29) for corner_data in corners_refined])
 
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)
 
# Calibrate camera using cv2.calibrateCamera()
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    chessboard_points,
    valid_corners,
    [imgs_gray[0].shape[1], imgs_gray[0].shape[0]],  # Image size (width, height)
    None,
    None
)

In [13]:
img = cv2.imread("data/ejemplo_figuras/pico_test.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
show_image(img_gray, "Grayscale Image")
write_image(img_gray, "data/ejemplo_figuras/pico_test_gray.jpg")

def compute_contours(img_gray):
    blurred_img = cv2.GaussianBlur(img_gray, (5, 5), 0)  # Suavizado gausiano
    show_image(blurred_img, "Blurred Image")
    write_image(blurred_img, "data/ejemplo_figuras/pico_test_blurred.jpg")
    img_thres = cv2.adaptiveThreshold(blurred_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    show_image(img_thres, "Thresholded Image")
    write_image(img_thres, "data/ejemplo_figuras/pico_test_thresholded.jpg")
    img_morph = cv2.morphologyEx(img_thres, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))  # Apertura morfológica
    show_image(img_morph, "Morphological Opening")
    write_image(img_morph, "data/ejemplo_figuras/pico_test_morphological.jpg")
    contours, _ = cv2.findContours(img_morph, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    return contours

compute_contours(img_gray)

(array([[[552, 817]],
 
        [[552, 819]],
 
        [[554, 819]],
 
        [[554, 817]]], dtype=int32),
 array([[[427, 811]],
 
        [[427, 812]],
 
        [[426, 813]],
 
        [[424, 813]],
 
        [[424, 815]],
 
        [[426, 815]],
 
        [[427, 814]],
 
        [[429, 814]],
 
        [[429, 811]]], dtype=int32),
 array([[[665, 805]],
 
        [[665, 807]],
 
        [[667, 807]],
 
        [[667, 805]]], dtype=int32),
 array([[[654, 805]],
 
        [[653, 806]],
 
        [[648, 806]],
 
        [[648, 808]],
 
        [[649, 808]],
 
        [[650, 809]],
 
        [[652, 809]],
 
        [[653, 808]],
 
        [[660, 808]],
 
        [[660, 806]],
 
        [[659, 805]]], dtype=int32),
 array([[[673, 804]],
 
        [[673, 806]],
 
        [[676, 806]],
 
        [[676, 804]]], dtype=int32),
 array([[[669, 804]],
 
        [[669, 806]],
 
        [[671, 806]],
 
        [[671, 804]]], dtype=int32),
 array([[[794, 803]],
 
        [[794, 806]],
 
        [[